<img src="http://oci02.img.iteso.mx/identidad_de_instancia_2018/ITESO/Logos%20ITESO/Logo-ITESO-Principal.jpg">

<center> <font color= #004A94> <font size = 6> Lic. Ingeniería Financiera </font> <br> <br> <font color= #047CFB> <font size = 4>I.F. Tamara Martínez - if709156@iteso.mx </font>

<center><font color= #004A94> <font size = 6> Microestructura y Sistemas de Trading </font> <br> <br>

<center><font color=#42c29b><font size=5> Laboratorio 0: Repaso de Python </font> <br> <br>

<center> <font color= #047CFB> <font size = 4> Primavera 2020 </font>  |  <font color= #047CFB> <font size = 4> Repositorio: <a href='https://github.com/tamaramtz/LAB_0_TVM.git'>Link</a></font>

---

# Resumen

En este laboratorio se generó nueva información de los precios a partir de los históricos en formato  OHLC (Open, High, Low, Close). Esta nueva información generada, la mayoría almacenada como nuevas columnas dentro del DataFrame que contiene los precios históricos OHLC, fue utilizada también como "variables explicativas" en modelos predictivos (ya sean de regresión o de clasificación). El enfóque de los cálculos que se realizaron para este laboratorio es "estadístico" utilizando información de las "velas".  También se graficó utilizando la librería de Plotly, todo esto utilizando una lógica de organizar el código con base a "funciones" y scripts separados (algo bastante útil para proyectos aplicados de python).

# Codigo

Importar las librerías utilizadas.

In [1]:
import funciones as fn                              # Para procesamiento de datos
import visualizaciones as vs                        # Para visualizacion de datos
import pandas as pd                                 # Procesamiento de datos
from datos import OA_Ak                             # Importar token para API de OANDA

ModuleNotFoundError: No module named 'funciones'

Descargamos los precios masivos de OANDA.

In [ ]:
OA_In = "EUR_USD"                                   # Instrumento
OA_Gn = "D"                                         # Granularidad de velas
fini = pd.to_datetime("2018-07-06 00:00:00").tz_localize('GMT')  # Fecha inicial
ffin = pd.to_datetime("2019-12-06 00:00:00").tz_localize('GMT')  # Fecha final
# Descargar precios masivos
df_pe = fn.f_precios_masivos(p0_fini=fini, p1_ffin=ffin, p2_gran=OA_Gn,
                             p3_inst=OA_In, p4_oatk=OA_Ak, p5_ginc=4900)

Grafica de los OHLC (Open, High, Low, Close) 

In [ ]:
vs_grafica1 = vs.g_velas(p0_de=df_pe.iloc[0:120, :])
vs_grafica1.show()

Se hace un conteo de velas.

In [ ]:

# multiplicador de precios
pip_mult = 10000

# -- 0A.1: Hora
df_pe['hora'] = [df_pe['TimeStamp'][i].hour for i in range(0, len(df_pe['TimeStamp']))]

# -- 0A.2: Dia de la semana.
df_pe['dia'] = [df_pe['TimeStamp'][i].weekday() for i in range(0, len(df_pe['TimeStamp']))]

# -- 0B: Boxplot de amplitud de velas (close - open).
df_pe['co'] = (df_pe['Close'] - df_pe['Open']) * pip_mult

# -- ------------------------------------------------------------ Graficar Boxplot plotly -- #
vs_grafica2 = vs.g_boxplot_varios(p0_data=df_pe[['co']], p1_norm=False)
vs_grafica2.show(

## Las funciones que se desarrollaron individualmente son las siguientes:

### Mes en el que ocurrió la vela.
Utilizando la columna de TimeStamp calcula el "Mes" en el que ocurrió la vela.

In [ ]:
# -- 01 Mes en el que ocurrió la vela.
df_pe['mes'] = [df_pe['TimeStamp'][i].month for i in range(0, len(df_pe['TimeStamp']))]

### Sesion de la vela
Sesión bursátil en la que ocurrió la vela,  el valor dentro de la columna deberás de colocarlo siguiendo la siguiente regla:

'asia':  si en la columna ['hora'] tiene alguno de estos valores -> 22, 23, 0, 1, 2, 3, 4, 5, 6, 7
'asia_europa': si en la columna ['hora'] tiene alguno de estos valores -> 8
'europa': si en la columna ['hora'] tiene alguno de estos valores -> 9, 10, 11, 12 
'europa_america': si en la columna ['hora'] tiene alguno de estos valores -> 13, 14, 15, 16
'america': si en la columna ['hora'] tiene alguno de estos valores -> 17, 18, 19, 20, 21

In [ ]:
for i in range(0, len(df_pe['hora'])):
    if df_pe['hora'][i] in [22, 23, 0, 1, 2, 3, 4, 5, 6, 7]:
        df_pe['sesion'] = 'asia'
    elif df_pe['hora'][i] in [8]:
        df_pe['sesion'] = 'asia_europa'
    elif df_pe['hora'][i] in [9, 10, 11, 12]:
        df_pe['sesion'] = 'europa'
    elif df_pe['hora'][i] in [13, 14, 15, 16]:
        df_pe['sesion'] = 'europa_america'
    elif df_pe['hora'][i] in [17, 18, 19, 20, 21]:
        df_pe['sesion'] = 'america'

### Amplitud de vela (en pips).
Calcular la diferencia entre las columnas ['Open'] y ['Close'], expresarla en pips.

In [ ]:
# -- 03 Amplitud de vela (en pips).
df_pe['oc'] = (df_pe['Open'] - df_pe['Close'])*pip_mult

### Amplitud de extremos (en pips).
Calcular la diferencia entre las columnas ['High'] y ['Low'], expresarla en pips.

In [ ]:
# -- 04 Amplitud de los extremos (en pips).
df_pe['hl'] = (df_pe['High'] - df_pe['Low'])*pip_mult

### Sentido de la vela (alcista o bajista)
En esta columna debes de asignarle el valor de 'alcista' para cuando ['Close'] >= ['Open'] y 'bajista' en el caso contrario.

In [ ]:
# -- 05 Sentido de la vela (alcista o bajista)
df_pe['sentido'] = ["alcista" if df_pe['Close'][i] >= df_pe['Open'][i] else "bajista"
                            for i in range(0, len(df_pe['Close']))]

### Conteo de velas consecutivas alcistas/bajistas.
En el DataFrame de los precios OHLC, para cada renglon, ir acumulando el valor de velas consecutivas ALCISTAS o BAJISTAS e ir haciendo el conteo de ocurrencia para cada caso. Se comienza el conteo a partir de la primera repetición, por ejemplo, ['sentido_c'] tendrá un 2  en el tiempo t cuando en el tiempo t-2 y tiempo t-1 haya sido el mismo valor que en el tiempo t. En este ejemplo ['sentido_c'] tendría un 2 (en el tiempo t-2 fue la primera vela, y la vela en tiempo t-1 y en tiempo t fueron 2 velas fueron consecutivamente en el mismo sentido)

In [ ]:
df_pe.loc[0,'conteo_c'] = 0 #primer dato de la columna
x=0
for i in range(1, len(df_pe['sentido'])):
    if df_pe['sentido'][i] == df_pe['sentido'][i-1]:
        x += 1
        df_pe.loc[i,'conteo_c'] = x
    else:
        x = 0
        df_pe.loc[i,'conteo_c'] = x


### Ventanas móviles de volatilidad
Utiliza la columna de ['hl'] como una medida de "volatilidad" en pips de las velas. Con esta columna, genera las siguientes columnas haciendo una "ventana móvil" del máximo de esos últimos n valores. Las columnas serán 3, una para cada valor de la "volatilidad móvil" para 5, 25 y 50 velas de histórico respectivamente.

['volatilidad_5']: Utilizando la información de las 5 anteriores velas.
['volatilidad_25']: Utilizando la información de las 25 anteriores velas.
['volatilidad_50']: Utilizando la información de las 50 anteriores velas.

In [ ]:
# -- 07 Ventanas móviles de volatilidad.
df_pe['volatilidad_5'] = df_pe.iloc[:, 11].rolling(window=5).mean() #volalitilidad con 5
df_pe['volatilidad_25'] = df_pe.iloc[:, 11].rolling(window=25).mean() #volalitilidad con 25
df_pe['volatilidad_50'] = df_pe.iloc[:, 11].rolling(window=50).mean() #volalitilidad con 50

### Gráfica con Plotly
Realiza una propuesta de gráfica utilizando alguna de las columnas que has generado y la librería plotly. Las reglas son las siguientes:

Tiene que tener título de gráfica
Tiene que tener título de eje x y etiquetas de eje x
Tiene que tener título de eje y y etiquetas de eje y
Se debe de poder visualizar una leyenda (en cualquier posición).

In [ ]:
# -- 08 Gráfica con Plotly
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_pe['TimeStamp'], y=df_pe['hl'], name="High-Low", line_shape='spline'))
fig.update_layout(title='Average High and Low prices',
                   xaxis_title='TimeStamp',
                   yaxis_title='Price')
fig.show()